In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Load and preprocess MNIST data
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model_name = 'LeNet'
dataset = 'MNIST'

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
teacher_model_path = '/content/base/LeNetMNIST_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha (1)

In [ ]:
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 0.0742 - accuracy: 0.9771
Test Loss: 0.07424774765968323, Test Accuracy: 0.9771000146865845
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0548 - accuracy: 0.9833
Test Loss: 0.05483292415738106, Test Accuracy: 0.983299970626831
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.9843
Test Loss: 0.048971135169267654, Test Accuracy: 0.9843000173568726
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0440 - accuracy: 0.9854
Test Loss: 0.04400905966758728, Test Accuracy: 0.9854000210762024
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0407 - accuracy: 0.9871
Test Loss: 0.04068867489695549, Test Accuracy: 0.9871000051498413
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0389 - accuracy: 0.9873
Test Loss: 0.03888789936900

In [ ]:
shutil.make_archive("student_same", 'zip', "student_same")
files.download("student_same.zip")

# Student 0.1 Alpha (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)

model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step - loss: 0.0748 - accuracy: 0.9778
Test Loss: 0.07480966299772263, Test Accuracy: 0.9778000116348267
Epoch 2:
313/313 [==============================] - 1s 5ms/step - loss: 0.0501 - accuracy: 0.9846
Test Loss: 0.05010770261287689, Test Accuracy: 0.9846000075340271
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 0.0408 - accuracy: 0.9868
Test Loss: 0.040816713124513626, Test Accuracy: 0.9868000149726868
Epoch 4:
313/313 [==============================] - 1s 2ms/step - loss: 0.0364 - accuracy: 0.9881
Test Loss: 0.03639257699251175, Test Accuracy: 0.988099992275238
Epoch 5:
313/313 [==============================] - 1s 2ms/step - loss: 0.0331 - accuracy: 0.9894
Test Loss: 0.03305165469646454, Test Accuracy: 0.9894000291824341
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 0.0308 - accuracy: 0.9893
Test Loss: 0.03079779632389

Zip data

In [ ]:
shutil.make_archive("student_same_2", 'zip', "student_same_2")
files.download("student_same_2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Student 0.1 Alpha (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)

model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 0.0770 - accuracy: 0.9758
Test Loss: 0.07702013850212097, Test Accuracy: 0.9757999777793884
Epoch 2:
313/313 [==============================] - 2s 5ms/step - loss: 0.0469 - accuracy: 0.9839
Test Loss: 0.04685361310839653, Test Accuracy: 0.9839000105857849
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 0.0380 - accuracy: 0.9872
Test Loss: 0.03800017014145851, Test Accuracy: 0.9872000217437744
Epoch 4:
313/313 [==============================] - 1s 2ms/step - loss: 0.0341 - accuracy: 0.9890
Test Loss: 0.034115079790353775, Test Accuracy: 0.9890000224113464
Epoch 5:
313/313 [==============================] - 2s 5ms/step - loss: 0.0314 - accuracy: 0.9895
Test Loss: 0.031365685164928436, Test Accuracy: 0.9894999861717224
Epoch 6:
313/313 [==============================] - 1s 5ms/step - loss: 0.0301 - accuracy: 0.9898
Test Loss: 0.030122121796

In [ ]:
shutil.make_archive("student_same_3", 'zip', "student_same_3")
files.download("student_same_3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Student same Alpha 0.5

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_50/'

train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 0.0749 - accuracy: 0.9769
Test Loss: 0.07488416135311127, Test Accuracy: 0.9768999814987183
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0564 - accuracy: 0.9821
Test Loss: 0.05644461140036583, Test Accuracy: 0.9821000099182129
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 0.0497 - accuracy: 0.9841
Test Loss: 0.049728453159332275, Test Accuracy: 0.9840999841690063
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0450 - accuracy: 0.9850
Test Loss: 0.04503950849175453, Test Accuracy: 0.9850000143051147
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0404 - accuracy: 0.9870
Test Loss: 0.04039209708571434, Test Accuracy: 0.9869999885559082
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.9869
Test Loss: 0.038291968405246735, Test Accuracy: 0.9868999719619751
Epoch 7:
313/313 [=====

In [ ]:
shutil.make_archive("student_same_50", 'zip', "student_same_50")
files.download("student_same_50.zip")

# Student 0.5 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'

train_custom(distiller, x_train,y_train,x_test,y_test)

shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
files.download("student_same_50_2.zip")

313/313 [==============================] - 1s 3ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 2ms/step - loss: 0.0775 - accuracy: 0.9750
Test Loss: 0.07745856791734695, Test Accuracy: 0.9750000238418579
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 0.9830
Test Loss: 0.04759855195879936, Test Accuracy: 0.9829999804496765
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.9869
Test Loss: 0.038304463028907776, Test Accuracy: 0.9868999719619751
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0344 - accuracy: 0.9881
Test Loss: 0.03440842777490616, Test Accuracy: 0.988099992275238
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0319 - accuracy: 0.9886
Test Loss: 0.03192931041121483, Test Accuracy: 0.9886000156402588
Epoch 6:
313/313 [==============================] - 2s 5ms/step - loss: 0.0302 - accuracy: 0.9898
Test Loss: 0.030151600018143654, Test Accuracy: 0.989799976348877
Epoch 7:
313/313 [=======

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# student_50 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'

train_custom(distiller, x_train,y_train,x_test,y_test)

shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
files.download("student_same_50_3.zip")

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 2s 4ms/step - loss: 0.0779 - accuracy: 0.9745
Test Loss: 0.07787740230560303, Test Accuracy: 0.9745000004768372
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0477 - accuracy: 0.9831
Test Loss: 0.047682225704193115, Test Accuracy: 0.9830999970436096
Epoch 3:
313/313 [==============================] - 1s 2ms/step - loss: 0.0387 - accuracy: 0.9864
Test Loss: 0.03870305046439171, Test Accuracy: 0.9864000082015991
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0343 - accuracy: 0.9881
Test Loss: 0.034302856773138046, Test Accuracy: 0.988099992275238
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 0.0322 - accuracy: 0.9890
Test Loss: 0.03222411498427391, Test Accuracy: 0.9890000224113464
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0308 - accuracy: 0.9903
Test Loss: 0.0307885222136

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Student same Alpha 0.9

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 4ms/step - loss: 0.0750 - accuracy: 0.9770
Test Loss: 0.07502587139606476, Test Accuracy: 0.9769999980926514
Epoch 2:
313/313 [==============================] - 1s 2ms/step - loss: 0.0547 - accuracy: 0.9826
Test Loss: 0.05474058911204338, Test Accuracy: 0.9825999736785889
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 0.9842
Test Loss: 0.047644395381212234, Test Accuracy: 0.9842000007629395
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0434 - accuracy: 0.9862
Test Loss: 0.04341650381684303, Test Accuracy: 0.9861999750137329
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0408 - accuracy: 0.9863
Test Loss: 0.04079493507742882, Test Accuracy: 0.986299991607666
Epoch 6:
313/313 [==============================] - 2s 5ms/step - loss: 0.0375 - accuracy: 0.9878
Test Loss: 0.037476472556591034, Test Accuracy: 0.9878000020980835
Epoch 7:
313/313 [======

In [ ]:
shutil.make_archive("student_same_90", 'zip', "student_same_90")
files.download("student_same.zip_90")

# Student 90 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
files.download("student_same_90_2.zip")

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 2s 4ms/step - loss: 0.0749 - accuracy: 0.9777
Test Loss: 0.07493124157190323, Test Accuracy: 0.9776999950408936
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0502 - accuracy: 0.9849
Test Loss: 0.050224658101797104, Test Accuracy: 0.9848999977111816
Epoch 3:
313/313 [==============================] - 2s 5ms/step - loss: 0.0406 - accuracy: 0.9871
Test Loss: 0.04056088998913765, Test Accuracy: 0.9871000051498413
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0363 - accuracy: 0.9882
Test Loss: 0.03632732108235359, Test Accuracy: 0.9882000088691711
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0332 - accuracy: 0.9892
Test Loss: 0.033225882798433304, Test Accuracy: 0.9891999959945679
Epoch 6:
313/313 [==============================] - 1s 5ms/step - loss: 0.0306 - accuracy: 0.9899
Test Loss: 0.030622247606515884, Test Accuracy: 0.9898999929428101
Epoch 7:
313/313 [====

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# student 90 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
files.download("student_same_90_3.zip")

313/313 [==============================] - 1s 3ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 4ms/step - loss: 0.0743 - accuracy: 0.9775
Test Loss: 0.07425890117883682, Test Accuracy: 0.9775000214576721
Epoch 2:
313/313 [==============================] - 1s 2ms/step - loss: 0.0497 - accuracy: 0.9846
Test Loss: 0.04969808831810951, Test Accuracy: 0.9846000075340271
Epoch 3:
313/313 [==============================] - 1s 2ms/step - loss: 0.0406 - accuracy: 0.9869
Test Loss: 0.04062308743596077, Test Accuracy: 0.9868999719619751
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0370 - accuracy: 0.9883
Test Loss: 0.03699176758527756, Test Accuracy: 0.9883000254631042
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0334 - accuracy: 0.9890
Test Loss: 0.03343488648533821, Test Accuracy: 0.9890000224113464
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0304 - accuracy: 0.9901
Test Loss: 0.030403781682252884, Test Accuracy: 0.9901000261306763
Epoch 7:
313/313 [======

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---